In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
dataset_train_path = '/Users/jonathanprasetyo/Documents/Snake Classifier/snake/train'
dataset_test_path = '/Users/jonathanprasetyo/Documents/Snake Classifier/snake/test'

# Define the input image dimensions
img_height, img_width = 224, 224
input_shape = (img_height, img_width, 3)

# Define the number of classes (snake species)
num_classes = len(os.listdir(dataset_train_path))

def create_snake_classifier_model():
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [4]:
# Use ImageDataGenerator for data augmentation
import pandas as pd
class_mapping_train = pd.read_csv('snake/Csv/train.csv', index_col=0)
class_mapping_val = pd.read_csv('snake/Csv/test.csv', index_col=0)


# Add '.jpg' to the 'UUID' column
# class_mapping_train['full_path'] = dataset_train_path + '/' + class_mapping_train['class_id'] + class_mapping_train['UUID']+ '.jpg'

# for i in class_mapping_train:
#   class_mapping_train['fullPath'][i] = dataset_train_path + '/' + str(class_mapping_train['class_id'][i]) + class_mapping_train['UUID'][i]+ '.jpg'



for index, row in class_mapping_train.iterrows():
  class_mapping_train.at[index, 'UUID'] = (
      dataset_train_path + '/' +
      str(row['class_id']) + '/' +
      row['UUID'] + '.jpg'
  )

for index, row in class_mapping_val.iterrows():
  class_mapping_val.at[index, 'UUID'] = (
      dataset_test_path + '/' +
      str(row['class_id']) + '/' +
      row['UUID'] + '.jpg'
  )


# Check for invalid image filenames
invalid_files = class_mapping_train[~class_mapping_train['UUID'].apply(os.path.exists)]

if not invalid_files.empty:
    print("Invalid image filenames:")
    print(invalid_files[['UUID']])
    print("These entries will be ignored.")

datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.3  # Split data into training and validation sets
    )

# train_generator = train_datagen.flow_from_directory(
#     dataset_train_path,
#     target_size=(img_height, img_width),
#     batch_size=64,
#     class_mode='categorical'
# )

train_generator = datagen.flow_from_dataframe(
        class_mapping_train,
        # directory=dataset_train_path,
        x_col='UUID',  # Assuming 'uuid' is the column for image filenames
        y_col='binomial',
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical',  # or 'binary' depending on your problem
        subset='training'  # Specify this is the training set
)

val_generator = datagen.flow_from_dataframe(
    class_mapping_val,
    # directory=dataset_test_path,
    x_col='UUID',
    y_col='binomial',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='validation'  # Specify this is the validation set
)


# val_generator = val_datagen.flow_from_directory(
#     dataset_train_path,
#     target_size=(img_height, img_width),
#     batch_size=64,
#     class_mode='categorical'
# )


# Create and compile the model
model = create_snake_classifier_model()

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=40,
    validation_data=val_generator
)

# Save the model
model.save('snake_classifier_model.h5')

Found 16672 validated image filenames belonging to 135 classes.
Found 941 validated image filenames belonging to 135 classes.
Epoch 1/40
260/260 [==============================] - 522s 2s/step - loss: 4.4938 - accuracy: 0.0287 - val_loss: 10.1831 - val_accuracy: 0.0000e+00
Epoch 2/40
260/260 [==============================] - 518s 2s/step - loss: 4.2047 - accuracy: 0.0621 - val_loss: 12.7631 - val_accuracy: 0.0000e+00
Epoch 3/40
260/260 [==============================] - 519s 2s/step - loss: 4.0102 - accuracy: 0.0915 - val_loss: 14.0758 - val_accuracy: 0.0000e+00
Epoch 4/40
260/260 [==============================] - 520s 2s/step - loss: 3.8188 - accuracy: 0.1219 - val_loss: 13.7680 - val_accuracy: 0.0000e+00
Epoch 5/40
260/260 [==============================] - 514s 2s/step - loss: 3.6261 - accuracy: 0.1521 - val_loss: 16.7161 - val_accuracy: 0.0000e+00
Epoch 6/40
260/260 [==============================] - 518s 2s/step - loss: 3.4225 - accuracy: 0.1902 - val_loss: 18.5407 - val_accurac

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 8))
plt.plot(model.history['accuracy'], label='accuracy')
plt.plot(model.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

plt.savefig("output_report.png")

NameError: name 'model' is not defined

<Figure size 800x800 with 0 Axes>